In [1]:
#Imports
from sqlalchemy import create_engine
import pandas as pd

In [2]:
#Setup connection to DB
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
#Print table names to verify connection
print(engine.table_names())

['Bookings', 'Facilities', 'Members']


In [4]:
#Connect to engine
con = engine.connect()

In [5]:
'''Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!'''
query = """
    SELECT f.name, SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS revenue  
    FROM `Facilities` AS f
    INNER JOIN `Bookings` AS b
    USING(facid)
    GROUP BY f.name 
    HAVING revenue < 1000
    ORDER BY revenue
"""
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [6]:
'''Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order'''
query = """
    SELECT DISTINCT m.firstname || ' ' || m.surname AS member, r.firstname || ' ' || r.surname AS recommender
    FROM `Members` AS m
    INNER JOIN `Members` as r
    ON m.recommendedby = r.memid
    ORDER BY m.surname, m.firstname
"""
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

                     member        recommender
0            Florence Bader    Ponder Stibbons
1                Anne Baker    Ponder Stibbons
2             Timothy Baker     Jemima Farrell
3                Tim Boothe         Tim Rownam
4            Gerald Butters       Darren Smith
5               Joan Coplin      Timothy Baker
6             Erica Crumpet        Tracy Smith
7                Nancy Dare    Janice Joplette
8           Matthew Genting     Gerald Butters
9                 John Hunt  Millicent Purview
10              David Jones    Janice Joplette
11            Douglas Jones        David Jones
12          Janice Joplette       Darren Smith
13           Anna Mackenzie       Darren Smith
14             Charles Owen       Darren Smith
15             David Pinker     Jemima Farrell
16        Millicent Purview        Tracy Smith
17         Henrietta Rumney    Matthew Genting
18         Ramnaresh Sarwin     Florence Bader
19               Jack Smith       Darren Smith
20          P

In [7]:
'''Q12: Find the facilities with their usage by member, but not guests'''
query = """
    SELECT firstname || ' ' || surname AS member, name AS facility, COUNT(facid) AS utilization
    FROM `Bookings` AS b
    INNER JOIN `Members` as m
    USING (memid)
    INNER JOIN `Facilities` as f
    USING (facid)
    WHERE memid != 0
    GROUP BY memid, facility
"""
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

            member         facility  utilization
0     Darren Smith  Badminton Court          132
1     Darren Smith   Massage Room 1           28
2     Darren Smith       Pool Table           28
3     Darren Smith    Snooker Table           12
4     Darren Smith     Squash Court           14
..             ...              ...          ...
197      John Hunt   Tennis Court 2            4
198  Erica Crumpet  Badminton Court            2
199  Erica Crumpet   Massage Room 1            2
200  Erica Crumpet     Table Tennis            2
201  Erica Crumpet   Tennis Court 1            1

[202 rows x 3 columns]


In [8]:
'''Q13: Find the facilities usage by month, but not guests'''
query = """
    SELECT strftime('%m', starttime) AS month, name AS facility, COUNT(facid) AS utilization
    FROM `Bookings` AS b
    INNER JOIN `Facilities` as f
    USING (facid)
    WHERE memid != 0
    GROUP BY month, facility
"""
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

   month         facility  utilization
0     07  Badminton Court           51
1     07   Massage Room 1           77
2     07   Massage Room 2            4
3     07       Pool Table          103
4     07    Snooker Table           68
5     07     Squash Court           23
6     07     Table Tennis           48
7     07   Tennis Court 1           65
8     07   Tennis Court 2           41
9     08  Badminton Court          132
10    08   Massage Room 1          153
11    08   Massage Room 2            9
12    08       Pool Table          272
13    08    Snooker Table          154
14    08     Squash Court           85
15    08     Table Tennis          143
16    08   Tennis Court 1          111
17    08   Tennis Court 2          109
18    09  Badminton Court          161
19    09   Massage Room 1          191
20    09   Massage Room 2           14
21    09       Pool Table          408
22    09    Snooker Table          199
23    09     Squash Court           87
24    09     Table Tennis

In [9]:
#Close engine connection
con.close()